# 基于langchain创建自己专属的对话大模型

1. 领域精准问答
2. 数据更新频繁
3. 生成内容可解释可追溯
4. 数据隐私保护

通过这个例子，我们将基于`LangChain`, `OpenAI(LLM)`,  `vector DB`构建一个属于自己的LLM模型。

主要使用的技术————***Retrieval Augmented Generation (RAG)***

首先确保自己拥有一个 `OpenAI API key` (也并非必须)


### 准备环境

In [1]:
! pip install -qU \
    langchain==0.0.316 \
    openai==0.28.1  \
    tiktoken==0.5.1  \
    cohere \
    chromadb==0.4.15

### 创建一个对话模型(no RAG)

In [ ]:
import os
from langchain.chat_models import ChatOpenAI

os.environ["OPENAI_API_KEY"] = "..."
os.environ["base_url"] = "..."

chat = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    openai_api_base=os.environ["base_url"],
    model='gpt-3.5-turbo'
)

OpenAI Python 的例子
```python
[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Knock knock."},
    {"role": "assistant", "content": "Who's there?"},
    {"role": "user", "content": "Orange."},
]
```
https://cookbook.openai.com/examples/how_to_format_inputs_to_chatgpt_models


但是langchain 需要使用以下的格式

In [17]:

from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)


messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Knock knock."),
    AIMessage(content="Who's there?"),
    HumanMessage(content="Orange"),
    
]

In [19]:
res = chat(messages)
res

AIMessage(content='Orange who?')

因为 `res`也是`AIMessage`属性，所以我们可以直接进行添加，即可实现下一次的响应

In [20]:
messages.append(res)
res = chat(messages)

print(res.content)

Orange you glad I'm here to assist you?


#### 处理LLM存在的缺陷
1. 容易出现幻觉
2. 信息滞后
3. 专业领域深度知识匮乏


In [ ]:
messages = [
    SystemMessage(content="你是一个专业的知识助手。"),
    HumanMessage(content="你知道baichuan2模型吗？"),
]

In [ ]:
res = chat(messages)
print(res.content)


是的，我了解baichuan2模型。baichuan2是一种基于深度学习的图像分割模型，主要用于图像语义分割任务。它采用了编码-解码的架构，通过卷积神经网络提取图像特征，并将特征映射到与输入图像大小相同的分割结果。baichuan2模型在许多图像分割比赛和任务中取得了很好的性能，并被广泛应用于医学图像分割、自动驾驶、图像编辑等领域。


chatgpt AI无法满足我们在某些特定领域的专业需求，我们可以通过知识注入的方式，利用prompt来解决这种问题：

In [ ]:

baichuan2_information = [
    "Baichuan 2是一个大规模多语言语言模型，它专注于训练在多种语言中表现优异的模型，包括不仅限于英文。这使得Baichuan 2在处理各种语言的任务时能够取得显著的性能提升。",
    "Baichuan 2是从头开始训练的，使用了包括了2.6万亿个标记的庞大训练数据集。相对于以往的模型，Baichuan 2提供了更丰富的数据资源，从而能够更好地支持多语言的开发和应用。",
    "Baichuan 2不仅在通用任务上表现出色，还在特定领域（如医学和法律）的任务中展现了卓越的性能。这为特定领域的应用提供了强有力的支持。"
]

source_knowledge = "\n".join(baichuan2_information)

In [ ]:
print(source_knowledge)


In [ ]:
query = "你知道baichuan2模型吗？"

prompt_template = f"""基于以下内容回答问题：

内容:
{source_knowledge}

Query: {query}"""

In [ ]:
prompt = HumanMessage(
    content=prompt_template
)
messages.append(prompt)

res = chat(messages)

In [ ]:
print(res.content)


当我们注入一些专业的知识后，模型就能够很好的回答相关问题。
如果每一个问题都去用相关的外部知识进行增强拼接的话，那么回答的准确性就大大增加？？？？

### 创建一个RAG对话模型

#### 1. 加载数据 （以baichuan2论文为例）

   https://arxiv.org/pdf/2309.10305v2.pdf

In [21]:
! pip install pypdf

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/


In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("https://arxiv.org/pdf/2309.10305.pdf")
pages = loader.load_and_split()

In [23]:
pages[0]

Document(page_content='Baichuan 2: Open Large-scale Language Models\nAiyuan Yang, Bin Xiao, Bingning Wang, Borong Zhang, Chao Yin, Chenxu Lv, Da Pan\nDian Wang, Dong Yan, Fan Yang, Fei Deng, Feng Wang, Feng Liu, Guangwei Ai\nGuosheng Dong, Haizhou Zhao, Hang Xu, Haoze Sun, Hongda Zhang, Hui Liu, Jiaming Ji\nJian Xie, Juntao Dai, Kun Fang, Lei Su, Liang Song, Lifeng Liu, Liyun Ru, Luyao Ma\nMang Wang, Mickel Liu, MingAn Lin, Nuolan Nie, Peidong Guo, Ruiyang Sun\nTao Zhang, Tianpeng Li, Tianyu Li, Wei Cheng, Weipeng Chen, Xiangrong Zeng\nXiaochuan Wang, Xiaoxi Chen, Xin Men, Xin Yu, Xuehai Pan, Yanjun Shen, Yaodong Yang\nYiding Wang, Yiyu Li, Youxin Jiang, Yuchen Gao, Yupeng Zhang, Zenan Zhou, Zhiying Wu\nBaichuan Inc.\nAbstract\nLarge language models (LLMs) have\ndemonstrated remarkable performance on\na variety of natural language tasks based\non just a few examples of natural language\ninstructions, reducing the need for extensive\nfeature engineering. However, most powerful\nLLMs are

#### 2. 知识切片 将文档分割成均匀的块。每个块是一段原始文本

In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50,
)

docs = text_splitter.split_documents(pages)

In [25]:
len(docs)



216

#### 3. 利用embedding模型对每个文本片段进行向量化，并储存到向量数据库中

In [28]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma


embed_model = OpenAIEmbeddings(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    openai_api_base=os.environ["base_url"]
)
vectorstore = Chroma.from_documents(documents=docs, embedding=embed_model , collection_name="openai_embed")

#### 4. 通过向量相似度检索和问题最相关的K个文档。

In [29]:
query = "How large is the baichuan2 vocabulary?"
result = vectorstore.similarity_search(query ,k = 2)

In [30]:
result

[Document(page_content='capabilities in other languages. This hinders the\ndevelopment and application of LLMs in specific\nlanguages, such as Chinese.\nIn this technical report, we introduce Baichuan\n2, a series of large-scale multilingual language\nmodels. Baichuan 2 has two separate models,\nBaichuan 2-7B with 7 billion parameters and\nBaichuan 2-13B with 13 billion parameters. Both\nmodels were trained on 2.6 trillion tokens, which\nto our knowledge is the largest to date, more than', metadata={'page': 1, 'source': 'C:\\Users\\kalin\\AppData\\Local\\Temp\\tmpchwtntxm\\tmp.pdf'}),
 Document(page_content='Baichuan 2: Open Large-scale Language Models\nAiyuan Yang, Bin Xiao, Bingning Wang, Borong Zhang, Chao Yin, Chenxu Lv, Da Pan\nDian Wang, Dong Yan, Fan Yang, Fei Deng, Feng Wang, Feng Liu, Guangwei Ai\nGuosheng Dong, Haizhou Zhao, Hang Xu, Haoze Sun, Hongda Zhang, Hui Liu, Jiaming Ji\nJian Xie, Juntao Dai, Kun Fang, Lei Su, Liang Song, Lifeng Liu, Liyun Ru, Luyao Ma\nMang Wang, Mic

#### 5. 原始`query`与检索得到的文本组合起来输入到语言模型，得到最终的回答

In [34]:
def augment_prompt(query: str):
  # 获取top3的文本片段
  results = vectorstore.similarity_search(query, k=3)
  source_knowledge = "\n".join([x.page_content for x in results])
  # 构建prompt
  augmented_prompt = f"""Using the contexts below, answer the query.

  contexts:
  {source_knowledge}

  query: 
  {query}
  """
  return augmented_prompt

In [35]:
print(augment_prompt(query))

Using the contexts below, answer the query.

  contexts:
  capabilities in other languages. This hinders the
development and application of LLMs in specific
languages, such as Chinese.
In this technical report, we introduce Baichuan
2, a series of large-scale multilingual language
models. Baichuan 2 has two separate models,
Baichuan 2-7B with 7 billion parameters and
Baichuan 2-13B with 13 billion parameters. Both
models were trained on 2.6 trillion tokens, which
to our knowledge is the largest to date, more than
Baichuan 2: Open Large-scale Language Models
Aiyuan Yang, Bin Xiao, Bingning Wang, Borong Zhang, Chao Yin, Chenxu Lv, Da Pan
Dian Wang, Dong Yan, Fan Yang, Fei Deng, Feng Wang, Feng Liu, Guangwei Ai
Guosheng Dong, Haizhou Zhao, Hang Xu, Haoze Sun, Hongda Zhang, Hui Liu, Jiaming Ji
Jian Xie, Juntao Dai, Kun Fang, Lei Su, Liang Song, Lifeng Liu, Liyun Ru, Luyao Ma
Mang Wang, Mickel Liu, MingAn Lin, Nuolan Nie, Peidong Guo, Ruiyang Sun
Baichuan 1 64,000 0.570
Baichuan 2 125,696 0

In [36]:
# 创建prompt
prompt = HumanMessage(
    content=augment_prompt(query)
)

messages.append(prompt)

res = chat(messages)

print(res.content)

The Baichuan 2 vocabulary size is 125,696.


### 没有OPENAI api key怎么办 创建一个非openai的对话模型  


1.   embedding模型  
2.   chat模型


In [37]:
! pip install sentence-transformers

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
     -- ------------------------------------- 0.5/10.5 MB 2.8 MB/s eta 0:00:04
     --- ------------------------------------ 0.8/10.5 MB 3.0 MB/s eta 0:00:04
     ------ --------------------------------- 1.6/10.5 MB 2.4 MB/s eta 0:00:04
     -------- ------------------------------- 2.1/10.5 MB 2.6 MB/s eta 0:00:04
     ---------- ----------------------------- 2.6/10.5 MB 2.6 MB/s eta 0:00:04
     ------------ --------------------------- 3.1/10.5 MB 2.7 MB/s eta 0:00:03
     --------------- ------------------------ 3.9/10.5 MB 2.8 MB/s eta 0:00:03
     ----------------- ---------------------- 4.5/10.5 MB 2.7 MB/s eta 0:00:03
     -------------------- ------------------- 5.2/10.5 MB 2.7 MB/s eta 0:00:02
     --------------------- ------------------ 5.5/10.5 MB 2.8 MB/s eta 0:00:02
     ----------------------- ---------------- 6.0/10.5 MB 2.6 MB/s eta 0:00:02

In [38]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

model_name = "sentence-transformers/sentence-t5-large"

In [39]:
embedding = HuggingFaceEmbeddings(model_name=model_name)

d:\miniconda3\envs\rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' pac

In [43]:
vectorstore_hf = Chroma.from_documents(documents=docs, embedding=embedding , collection_name="huggingface_embed")

In [44]:
result = vectorstore_hf.similarity_search(query ,k = 2)


In [45]:
print(result)


[Document(page_content='a high compression rate for efficient inference,\nand an appropriately sized vocabulary to ensure\nadequate training of each word embedding. We\nhave taken both these aspects into account. We\nhave expanded the vocabulary size from 64,000\nin Baichuan 1 to 125,696, aiming to strike a\nbalance between computational efficiency and\nmodel performance.\nTokenizer V ocab Size Compression Rate ↓\nLLaMA 2 32,000 1.037\nBloom 250,680 0.501\nChatGLM 2 64,794 0.527\nBaichuan 1 64,000 0.570\nBaichuan 2 125,696 0.498', metadata={'page': 2, 'source': 'C:\\Users\\kalin\\AppData\\Local\\Temp\\tmpchwtntxm\\tmp.pdf'}), Document(page_content='a high compression rate for efficient inference,\nand an appropriately sized vocabulary to ensure\nadequate training of each word embedding. We\nhave taken both these aspects into account. We\nhave expanded the vocabulary size from 64,000\nin Baichuan 1 to 125,696, aiming to strike a\nbalance between computational efficiency and\nmodel perfo

通过本地部署的模型进行交互